In [ ]:
from IPython.display import display
import ipywidgets
import ipywidgets.widgets as widgets
import traitlets
from jetbot import Robot, Camera, bgr8_to_jpeg
from SCSCtrl import TTLServo

import threading

import paho.mqtt.client as mqtt
from datetime import datetime
import pytz
import time
import json
import random as rd

In [22]:
robot = Robot()
camera = Camera()

In [24]:
# 한국 시간대 (Asia/Seoul)로 설정
korea_timezone = pytz.timezone("Asia/Seoul")

#Broker IP Address 와 Port
#라즈베리파이5의 IP 주소로 수정필요
address = "172.20.10.6"
port = 1883

commandTopic = "AGV/command"
sensingTopic = "AGV/sensing"
sensingData = {
    "time" : "None",
    "num1": 0.15,
    "num2": 0.99,
    "is_finish": 0,
    "manual_mode" : "off"
}

publishingData = None

In [25]:
def agv_stop():
    robot.stop()
    
def agv_forward():
    robot.forward(0.4)

def agv_backward():
    robot.backward(0.4)

def agv_left():
    robot.left(0.3)

def agv_right():
    robot.right(0.3)

In [26]:
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("connected OK")
    else:
        print("Bad connection Returned code=", rc)

def on_publish(client, userdata, result):
    print("data published")

def on_message(client, userdata, msg):
    global message
    message = json.loads(msg.payload.decode("utf-8"))
    #print(message, type(message))
    
    commandlbl.value = message["cmd_string"]
    if message["cmd_string"] == "go":      agv_forward()
    elif message["cmd_string"] == "mid":   agv_stop()
    elif message["cmd_string"] == "left":  agv_left()
    elif message["cmd_string"] == "right": agv_right()
    elif message["cmd_string"] == "back":  agv_backward()
    elif message["cmd_string"] == "stop":  print('stop')
    elif message["cmd_string"] == "exit":
        print('exit')
        publishingData.stop()
        agv_stop()

In [27]:
class sensorReadPublish(threading.Thread):
    
    def __init__(self):
        super().__init__()
        self.th_flag = True
        
    def run(self):

        while self.th_flag:
            #소수점 둘째자리까지 생성
            num1 = round(rd.random(),2)
            num2 = round(rd.random(),2)
            
            #Label widget 에 출력하기 위해 str 형변환
            num1lbl.value = str(num1)
            num2lbl.value = str(num2)
            
            #현재 시간 정보 가져오기
            current_time = datetime.now(korea_timezone)
            sensingData["time"] = current_time.strftime("%Y-%m-%d %H:%M:%S")
            sensingData["num1"] = num1
            sensingData["num2"] = num2
            
            #GUI Controller 로 publish 하
            client.publish(sensingTopic, json.dumps(sensingData), 1)
            time.sleep(0.5)
        
    def stop(self):
        self.th_flag = False


In [28]:
client = None
def startPub(change):
    global publishingData, startBtn, client
    
    if startBtn.description == "Start" :
        #MQTT Client 객체 생성
        client = mqtt.Client()
        #Broker 연결
        client.connect(address, port)
        
        #Callback 함수 바인딩
        client.on_connect = on_connect
        client.on_publish = on_publish
        
        #loop_start() 를 이용해 비동기적으로 MQTT Protocol 동작
        client.loop_start()
        
        #sensorReadPublish() 객체 생성
        publishingData = sensorReadPublish()
        #thread 시작
        publishingData.start()
        
        #Start 버튼 정보 변경
        startBtn.description = "Stop"
        startBtn.button_style = 'danger'
        
        #Manual 버튼 활성화
        manualBtn.disabled = False
        
    elif startBtn.description == "Stop" :
    
        #thread 종료
        publishingData.stop()
        
        startBtn.description = "Start"
        startBtn.button_style = 'info'
        #print("Stop", client.is_connected())

startBtn.on_click(startPub)

In [29]:
def startSub(change):
    global manualBtn, client
    
    if manualBtn.description == "Manual" :
        #MQTT Subscribe 시작, topic, qos
        client.subscribe(commandTopic, 1)
        #on_message callback 함수 등록
        client.on_message = on_message
        
        manualBtn.description = "Auto"
        manualBtn.button_style = 'success'
        
    elif manualBtn.description == "Auto" :
        #MQTT Publish 종료
        publishingData.stop()
        manualBtn.description = "Manual"
        manualBtn.button_style = 'warning'
        manualBtn.disabled = True

manualBtn.on_click(startSub)

In [20]:
time.sleep(0.1)
robot.stop()
camera.stop()
print('End')

End
